<a href="https://colab.research.google.com/github/jinwoongokRe/AI_basic_study/blob/master/stock_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import pandas as pd